# Non-daily #1589 
* Need Monday and Wednesday data from April 2025. 

In [1]:
import geopandas as gpd
import google.auth
import merge_operator_service
import pandas as pd
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
credentials, project = google.auth.default()

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
ROUTE_DIR_MONTH_FILE = GTFS_DATA_DICT.digest_tables.monthly_route_schedule_vp_report

In [5]:
route_dir_month_df = pd.read_parquet(f"{RT_SCHED_GCS}{ROUTE_DIR_MONTH_FILE}.parquet")

In [6]:
wed_date = ("2025-04-16",)  # main April date

In [7]:
mon_date = "2025-04-14"

In [8]:
SCHEDULED_SERVICES_FILE = GTFS_DATA_DICT.digest_tables.scheduled_service_hours

In [9]:
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

In [14]:
RT_SCHED_GCS

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/'

In [10]:
GTFS_DATA_DICT.rt_vs_schedule_tables.sched_route_direction_metrics

'schedule_route_dir/schedule_route_direction_metrics'

In [15]:
wed_url = "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/schedule_route_dir/schedule_route_direction_metrics_2025-04-16.parquet"

In [18]:
wed_df = gpd.read_parquet(
    wed_url,
    storage_options={"token": credentials.token},
).drop(columns = ["geometry"])

In [19]:
wed_df.head()

,schedule_gtfs_dataset_key,route_id,direction_id,common_shape_id,avg_scheduled_service_minutes,route_primary_direction,avg_stop_miles,n_trips,time_period,frequency
0,cb8a465cffec67c8fd90f31b389ed4c3,76991,0.00,p_1437167,20.00,Westbound,3.04,8,all_day,0.33
1,cb8a465cffec67c8fd90f31b389ed4c3,76991,0.00,p_1437167,20.00,Westbound,3.04,8,offpeak,0.50
2,330fd5b796496eddf2ce1d1b9828961d,10986,0.00,p_1433090,50.44,Northbound,12.11,9,all_day,0.38
3,330fd5b796496eddf2ce1d1b9828961d,10986,0.00,p_1433090,50.44,Northbound,12.11,6,offpeak,0.38
4,330fd5b796496eddf2ce1d1b9828961d,10986,0.00,p_1433090,50.44,Northbound,12.11,3,peak,0.38


In [20]:
mon_url = "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/schedule_route_dir/schedule_route_direction_metrics_2025-04-14.parquet"

In [21]:
mon_df = gpd.read_parquet(
    mon_url,
    storage_options={"token": credentials.token},
).drop(columns = ["geometry"])

In [ ]:
mon_df.head